The Gans Project by Balu, Marvin, Frederico and Joachim 

The first task is to get the lat., long., population, country, countrycode and wiki_dataid for a list of cities in Europe and put it in a dataframe

In [25]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
import re
import time
#!pip install python-dotenv
#%load_ext dotenv
#%dotenv
import os


API_key = os.getenv("API_K_C") #gets the API key from .env
#define the list of cities
cit = ['Berlin', 'London', 'Paris', 'Rome', 'Wien','Tallinn', 'Budapest', 'Warschau', 'Prague', 'Lisbon', 'Madrid', 'Amsterdam', 'Zagreb', 'Sofia', 'Athens']# use Englisch if possible
citd =['Berlin', 'Munich', 'Frankfurt', 'Kiel', 'Rostock', 'Leipzig', 'Cologne', 'Dresden', 'Erfurt', 'Jena', 'Weimar', 'Essen', 'Stuttgart', 'Mainz', 'Hannover', 'Hamburg']
#strategy is to first get the wikidataid and than collect the data from geoDB; openweather has no population data or at least I coul´nt see them 

cities_id = [] # initiate an empty id list
dfList = []
def demo(cities):
    cities_id = [] # initiate an empty id list
    dfList = []
    for city in cities:
        #retrieve the wikidataId
        time.sleep(1)
        url1 = 'https://en.wikipedia.org/wiki/{}'.format(city) #go to the wiki site of the city
        citem = requests.get(url1, 'html.parser') # get the html
        if BeautifulSoup(citem.content) != None:
            soup = BeautifulSoup(citem.content)# soup the content
        if soup.find('li', {'id':'t-wikibase'}).find('a')['href'] != None:
            wikidata_link = soup.find('li', {'id':'t-wikibase'}).find('a')['href'] #find the link that contains the wikidataid e.g. London https://www.wikidata.org/wiki/Q84
        #wl.append(wikidata_link)
        # \d+ is a regular expression and means one digit or more, the wiki data id consist of a Q followed by severaldigits
        #for group() in re see: https://www.tutorialspoint.com/What-is-the-groups-method-in-regular-expressions-in-Python
        city_id = re.search('Q\d+', wikidata_link).group()
        cities_id.append(city_id)
        #use the wikidataId to retrieve infrormation from geoDB
        url2 = "https://wft-geo-db.p.rapidapi.com/v1/geo/cities/{}".format(city_id)
        headers = {
        "X-RapidAPI-Key": API_key,
        "X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com"
        }    
        response = requests.request("GET", url2, headers=headers)# gets a json-like string fromg geoDB containing the necessary informations
        print(response.json())
        cit_dic = {}#make a dictionary to retrieve the information
        cit_dic['City'] = response.json()['data']['name']
        cit_dic['Country'] = response.json()['data']['country']
        cit_dic['CountryCode'] = response.json()['data']['countryCode']
        cit_dic['WikiDataId'] = response.json()['data']['wikiDataId']
        cit_dic['Latitude'] = round(response.json()['data']['latitude'], 4)
        cit_dic['Longitude'] = round(response.json()['data']['longitude'], 4)
        cit_dic['Population'] = response.json()['data']['population']
        cit_dic['Timezone'] = response.json()['data']['timezone']
        
        dfList.append(cit_dic) #put it in a list
        df_demo = pd.DataFrame(dfList) # transform the list to df
        #print(dfList)
    return df_demo
demo(citd) #run the whole thing - good luck!

{'data': {'id': 3453309, 'wikiDataId': 'Q64', 'type': 'CITY', 'city': 'Berlin', 'name': 'Berlin', 'country': 'Germany', 'countryCode': 'DE', 'region': 'Berlin', 'regionCode': 'BE', 'elevationMeters': None, 'latitude': 52.516666666, 'longitude': 13.383333333, 'population': 3664088, 'timezone': 'Europe__Berlin', 'distance': None, 'deleted': False, 'placeType': 'CITY'}}
{'data': {'id': 45633, 'wikiDataId': 'Q84', 'type': 'CITY', 'city': 'London', 'name': 'London', 'country': 'United Kingdom', 'countryCode': 'GB', 'region': 'England', 'regionCode': 'ENG', 'elevationMeters': 36, 'latitude': 51.507222222, 'longitude': -0.1275, 'population': 8908081, 'timezone': 'Europe__London', 'distance': None, 'deleted': False, 'placeType': 'CITY'}}
{'data': {'id': 144571, 'wikiDataId': 'Q90', 'type': 'CITY', 'city': 'Paris', 'name': 'Paris', 'country': 'France', 'countryCode': 'FR', 'region': 'Île-de-France', 'regionCode': 'IDF', 'elevationMeters': 28, 'latitude': 48.856944444, 'longitude': 2.351388888, 

,City,Country,CountryCode,WikiDataId,Latitude,Longitude,Population,Timezone
0,Berlin,Germany,DE,Q64,52.5167,13.3833,3664088,Europe__Berlin
1,London,United Kingdom,GB,Q84,51.5072,-0.1275,8908081,Europe__London
2,Paris,France,FR,Q90,48.8569,2.3514,2165423,Europe__Paris
3,Rome,Italy,IT,Q220,41.8931,12.4828,2872800,Europe__Rome
4,Vienna,Austria,AT,Q1741,48.2083,16.3725,1911191,Europe__Vienna
5,Tallinn,Estonia,EE,Q1770,59.4372,24.7450,438341,Europe__Tallinn
6,Budapest,Hungary,HU,Q1781,47.4983,19.0408,1723836,Europe__Budapest
7,Warsaw,Poland,PL,Q270,52.2300,21.0111,1790658,Europe__Warsaw
8,Prague,Czech Republic,CZ,Q1085,50.0880,14.4208,1301432,Europe__Prague
9,Lisbon,Portugal,PT,Q597,38.7167,-9.1333,506654,Europe__Lisbon
